In [1]:
import json
import os
import subprocess
from IPython.display import clear_output
import torch.optim as optim

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

device is cuda


# Generate metrics

In [2]:
from functions_utils_pomo import get_metrics     

In [3]:
cur_path = os.getcwd()
metrics = get_metrics(cur_path)
os.chdir('./Reward_weights')
for m in metrics:

    dic_tarif_sent_disp = {'v_required': 0,
                    'v_sent': 0,
                    'v_sent_full':0,
                    'v_degraded':0,
                    'cancelled':0, #cancel departure
                    'function_not_found':0,
                    'v1_not_sent_from_s1':0,
                    'v3_not_sent_from_s3':0,
                    'v_not_found_in_last_station':0,
                    'ff_required':0,
                    'ff_sent':0,
                    'rupture_ff':0,       
                    'z1_VSAV_sent': 0,
                    'z1_FPT_sent': 0,
                    'z1_EPA_sent': 0,
                     'VSAV_needed':0,
                     'FPT_needed':0,
                     'EPA_needed':0,
                     'VSAV_disp':0,
                     'FPT_disp':0,
                     'EPA_disp':0,
                    'skill_lvl':0
                    } 

    dic_tarif_sent_disp[m] = -100

    if m == 'v_degraded':
        
        dic_tarif_sent_disp['v_sent_full'] = 10


    with open(f"rw_"+ m +"_r100_cf3.json", "w") as f:
        json.dump(dic_tarif_sent_disp, f)

os.chdir("../")

In [5]:
os.chdir('./Reward_weights')
metric = "v1_not_sent_from_1st_station"
with open(f"rw_{metric}_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_sent': 0,
 'v_sent_full': 0,
 'v_degraded': 0,
 'cancelled': 0,
 'function_not_found': 0,
 'v1_not_sent_from_1st_station': -100,
 'v_not_found_in_last_station': 0,
 'z1_sent': 0,
 'skill_lvl': 0}

# Agent params POMO

In [6]:
path = "./SVG_model/shared_state_pomo.pt"
data = torch.load(path, map_location=device,weights_only=False)
print("Type :", type(data))
print(data.shape)
print(data)

Type : <class 'numpy.ndarray'>
(82, 40)
[[0.93103448 1.         0.88888889 ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 [0.125      0.125      0.125      ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
for metric in metrics:
    file_path = f"Reward_weights/rw_pomo_agent_{metric}_r100_cf3.json"
    try: os.remove(file_path)
    except FileNotFoundError: print(f"File '{file_path}' not found.")
print()

#-------Start
intervalle = 40
start = 1

while start <= 53080:
    
    end = start + intervalle - 1
    if end > 53080: end = 53080
    cmd = ["python3", "-u", "agent_pomo_run.py", "--start", str(start), "--end", str(end)]

    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    line_count = 0
    for line in process.stdout:
        line_count += 1
        #if line_count % 100 == 0:
           #clear_output(wait=True)

        print(line.strip())
    
    process.wait()
    
    start += intervalle
    

I copy-pasted the output of previous block in the log.txt file so we can load this ipynb file in github

In [8]:
path = "./SVG_model/shared_state_pomo.pt"
data = torch.load(path, map_location=device,weights_only=False)
print("Type :", type(data))
print(data.shape)
print(data)

Type : <class 'numpy.ndarray'>
(82, 40)
[[1.         1.         0.85185185 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.125      0.         0.5        ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# Reward  obtenu après itération

In [12]:
os.chdir('./Reward_weights')
metric = "v_degraded"
with open(f"rw_pomo_agent_{metric}_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

{'v_required': 55,
 'v_sent': 56,
 'v_sent_full': 55,
 'v_degraded': 1,
 'rupture_ff': 6,
 'function_not_found': 5,
 'v1_not_sent_from_s1': 11,
 'v3_not_sent_from_s3': 0,
 'v_not_found_in_last_station': 0,
 'ff_required': 0,
 'ff_sent': 200,
 'z1_VSAV_sent': 0,
 'z1_FPT_sent': 0,
 'z1_EPA_sent': 0,
 'VSAV_needed': 0,
 'FPT_needed': 0,
 'EPA_needed': 0,
 'VSAV_disp': 8,
 'FPT_disp': 0,
 'EPA_disp': 0,
 'skill_lvl': 331.0}

# Reward Mean

In [3]:
os.chdir('./Reward_weights')
metric = "v_degraded"
with open(f"rw_mean_pomo_agent_{metric}_r100_cf3.json", "r") as f:
    d = json.load(f)
os.chdir("../")
d

682.99

# NEW version POMO test

In [7]:
from agent import POMO_Agent

def generate_instances(batch_size, n_nodes, device):
    return torch.rand(batch_size, n_nodes, 2, device=device)

def compute_tour_length(coords, tours):
    # coords: [B, N, 2], tours: [B, pomo, N]
    B, P, N = tours.size()
    tour_coords = torch.gather(coords.unsqueeze(1).expand(-1, P, -1, -1), 2, tours.unsqueeze(-1).expand(-1, -1, -1, 2))
    rolled = torch.roll(tour_coords, shifts=-1, dims=2)
    lengths = ((tour_coords - rolled) ** 2).sum(dim=-1).sqrt().sum(dim=-1)  # [B, P]
    return lengths

N_EPOCHS = 200
BATCH_SIZE = 256
N_NODES = 80
POMO_SIZE = 8

hyper_params = {"state_size":3280, "action_size":80, "layer_size":1024, "num_layers":8, "use_batchnorm":0, "device":device, "seed":41}
model = POMO_Agent(**hyper_params)
network = model.network
for epoch in range(N_EPOCHS):
    coords = generate_instances(BATCH_SIZE, N_NODES, device)
    B = coords.size(0)

    node_embeds = network.encoder(coords)  # [B, N, H]
    rewards = []
    log_probs = []

    for _ in range(POMO_SIZE):
        mask = torch.zeros(B, N_NODES, device=device)
        tour = []
        log_p = []
        for _ in range(N_NODES):
            probs = network(node_embeds, mask)
            m = torch.distributions.Categorical(probs)
            action = m.sample()
            mask[torch.arange(B), action] = 1
            tour.append(action)
            log_p.append(m.log_prob(action))
        tour = torch.stack(tour, dim=1)  # [B, N]
        log_p = torch.stack(log_p, dim=1)  # [B, N]
        log_probs.append(log_p.sum(dim=1))  # [B]
        rewards.append(compute_tour_length(coords, tour.unsqueeze(1)).squeeze(1))  # [B]

    log_probs = torch.stack(log_probs, dim=1)  # [B, P]
    rewards = torch.stack(rewards, dim=1)  # [B, P]

    # baseline POMO: moyenne sur chaque instance
    baseline = rewards.mean(dim=1, keepdim=True)
    advantage = baseline - rewards  # minimiser la distance => max(-length)
    loss = (advantage.detach() * log_probs).mean()

    model.optimizer.zero_grad()
    loss.backward()
    model.optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Avg reward: {rewards.min(dim=1)[0].mean():.4f}")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)